In [20]:
import pandas as pd
import jsonlines
from typing import List, Optional
import sys
sys.path.append("../")
from src.engine.embedding import Embedding

In [2]:
data = pd.read_parquet("../data/yappy_hackaton_2024_40k_v1_to_elastic.pq")

In [3]:
embedding = Embedding(device="cuda:0")

In [4]:
concat_fields = list("passage: " + data["description_postprocessed"] + " " + data["title_shazam_postprocessed"] + " " + data["subtitle_shazam_postprocessed"])
# concat_fields = concat_fields[:1000]

In [5]:
bs = 500
text_embeddings: List[List[float]] = []
for i in range(0, len(concat_fields), bs):
    batch = concat_fields[i:i+bs]
    text_embeddings.extend(embedding(texts=batch))

In [6]:
data["embedding"] = text_embeddings

In [21]:
def create_documents_jsonl(data: Optional[pd.DataFrame] = None, path_to_pq: Optional[str] = None, path_to_save: str = "../data/documents.jsonl"):
    if data is None and path_to_csv is None:
        raise Exception
    if data is None:
        data = pd.read_parquet(path_to_pq)
        
    with jsonlines.open(path_to_save, mode="a") as writer:
        for idx, row in data.iterrows():
            sample = {}
            sample["doc_id"] = row["index orig"]
            sample["embedding"] = row["embedding"]
            sample["text_hashtags"] = row["description_postprocessed"]
            sample["video_hastags"] = ""
            sample["audio_transcription"] = row["transcription_postprocessed"]
            sample["song_name"] = row["title_shazam_postprocessed"]
            sample["song_author"] = row["subtitle_shazam_postprocessed"]
            writer.write(sample)

NameError: name 'data' is not defined

In [22]:
create_documents_jsonl(data=data, path_to_save="../data/documents.jsonl")

## Indexing documents.jsonl

In [1]:
import sys
import os
sys.path.append("../")
from src.index import index_jsonl_documents, create_index
from src.elastic.elastic_api import ElasticIndex

In [2]:
os.environ["NAME"]="team_lyam"
os.environ["ELASTIC_PORT"]="8201"
os.environ["INDEX_NAME"]="prod"
os.environ["ELASTIC_PASSWORD"]="IqNY6K=Sp7cKuLIvOwiZ"

In [5]:
create_index(elastic_ca_certs_path="../src/elastic/certs/http_ca.crt", path_to_index_json="../src/elastic/settings/index.json")

2024-06-11 01:05:31.203 | INFO     | src.elastic.elastic_api:create_index:31 - Index with name 'prod' is created.


In [6]:
index_jsonl_documents(elastic_ca_certs_path="../src/elastic/certs/http_ca.crt", path_to_jsonl="../data/documents.jsonl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40000/40000 [06:33<00:00, 101.64docs/s]
2024-06-11 01:12:12.866 | INFO     | src.elastic.elastic_api:count_documents_in_index:49 - Count of documents: 40000


In [3]:
elastic_api = ElasticIndex(index_name=os.environ["INDEX_NAME"], 
                           elastic_host_port=os.environ["ELASTIC_PORT"], 
                           elastic_password=os.environ["ELASTIC_PASSWORD"], 
                           elastic_ca_certs_path="../src/elastic/certs/http_ca.crt")

In [4]:
elastic_api.delete_index()

In [7]:
elastic_api.count_documents_in_index()

2024-06-11 01:12:16.698 | INFO     | src.elastic.elastic_api:count_documents_in_index:49 - Count of documents: 40000


In [17]:
create_index(elastic_ca_certs_path="../src/elastic/certs/http_ca.crt", path_to_index_json="../src/elastic/settings/index.json")

2024-06-11 00:55:23.670 | INFO     | src.elastic.elastic_api:create_index:31 - Index with name 'prod' is created.


In [9]:
import jsonlines

def _generate_documents(path_to_documents: str):
    with jsonlines.open(path_to_documents) as reader:
        for i, document in enumerate(reader):
            yield document

for doc in _generate_documents("../data/documents.jsonl"):
    print(doc)
    break

{'doc_id': 0, 'embedding': [0.012173717841506004, 0.04491748288273811, -0.021984530612826347, 0.008221407420933247, 0.013345304876565933, -0.017649924382567406, -0.012340077199041843, -0.03648685663938522, 0.053175367414951324, 0.02182881534099579, -0.01628965325653553, 0.0024353505577892065, 0.1251152902841568, 0.0456862710416317, -0.05866904556751251, -0.02413851022720337, 0.008735253475606441, -0.014194724150002003, 0.03164525330066681, 0.006166752427816391, 0.03603892773389816, -0.04098595678806305, 0.05381198972463608, -0.015110117383301258, 0.02360347844660282, -0.04081352800130844, 0.024280942976474762, 0.050579220056533813, -0.0368955098092556, 0.03146945312619209, 0.027424674481153488, -0.04658631235361099, 0.010170988738536835, 0.0037284637801349163, 0.041366077959537506, 0.06404262781143188, -0.0208172295242548, 0.000562087690923363, 0.026825090870261192, 0.00561721995472908, 0.027679303660988808, 0.03853701427578926, 0.004646684508770704, -0.0283034797757864, 0.029875935986